<a href="https://colab.research.google.com/github/jungyoojang/MachineLearning/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트(validation set)

모델을 만들고 나서 테스트 세트는 딱 한 번 사용되는 것이 좋음

따라서 추가로 검증 세트를 또 나눔

대학 훈련 세트 60%, 검증 세트 20%, 테스트 세트 20%

In [3]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [7]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [8]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [9]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


훈련 세트와 검증 세트의 평가 점수를 이용하여 최적화

최적화 완료 후 테스트 세트로 모델을 최종 평가

In [10]:
print(dt.score(test_input, test_target))

0.8569230769230769


**검증 세트의 문제점?**

검증 세트로 인해 훈련 세트가 줄어듬

보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어짐

검증 세트를 너무 조금 떼어 놓으면 검증 점수가 불안정함

이러한 문제를 보완할 방법이 필요



## 교차 검증(cross vaildation)

검증 세트를 떼어 내어 평가하는 과정을 여러번 반복

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00931716, 0.00749564, 0.00773239, 0.00731683, 0.00710797]), 'score_time': array([0.00109315, 0.00111032, 0.00101209, 0.00106931, 0.00115085]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


교차 검증의 최종 점수

이름은 test_score이지만 테스트 점수가 아닌 검증 점수임

In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


**주의할 점**

cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음

만약 train_test_split()를 하지 않았다면 분할기를 지정해야함

cross_validate()는 회귀 모델일 경우 kFold 분할기를 사용하고 분류 모델일 경우 StratfiedKFold를 사용

섞지 않은 것이 디폴트이므로 따로 설정해줘야함

In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


훈련 세트를 먼저 섞은 후 10-폴드 교차 검증을 수행

In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


cross_validate는 자체적으로 모델을 훈련시키기는 하나 모델 파라미터나 하이터 파라미터를 찾는 것이 아니라 모델의 점수 검증을 위해 사용함

따라서 훈련(fit) 필수

교차 검증 아이디어를 활용하여 하이퍼파라미터 튜닝을 수행

매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾는 과정에서 테스트 세트를 사용하지 않고 교차 검증을 수행



## 하이퍼파라미터 튜닝

모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터



**그리드 서치(grid search)**

하이퍼파라미터 탐색과 교차 검증을 한 번에 수행

즉, cross_validate()함수를 호출할 필요가 없음



---

예제: 결정 트리 모델에서 min_impurity_decrease 최적값 찾기

GridSearchCV 클래스를 import 하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

GrideSearchCV의 cv 매개변수 기본값은 5이므로 params에서 설정한 5개의 min_impurity_decrese 별로 결정 트리를 5번 실행

즉, 25번의 결정 트리 모델을 훈련

n_jobs: 병렬 실행에서 사용할 CPU 코어 수이며, -1이면 모든 코어를 사용함.

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


검증 검수가 가장 높은 매개변수 조합의 모델은 gs.best_estimator_속성에 저장됨

In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수

In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


넘파이 argmax()함수를 사용하여 가장 큰 값의 인덱스를 추출

params 키에 저장된 매개변수 출력

In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


**그리드 서치를 이용한 최적화 과정 정리**

1. 탐색할 매개변수 지정
2. 훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾음  (이 조합은 그리드 서치 객체에 저장)
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련 (이 조합은 그리드 서치 객체에 저장)

예제: 결정 트리모델에서 min_impurity_decrease, max_depth, min_samples_split 최적값 찾기

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최상의 교차 검증 점수 확인

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


찾은 파라미터로 모델을 생성하고 테스트 점수 확인

In [ ]:
dt = gs,best_estimator_


### 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 감격을 미리 정하기 어려울 수 있음

이럴때 랜덤 서치를 사용하면 좋음

매개변수 값의 목록이 아닌 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달함

**싸이파이 scipy**

파이썬의 핵심 과학 라이브러리 중 하나

stats 서브 패키지 사용

통계학과 관련된 패키지

uniform과 randint 클래스

In [ ]:
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 7, 6, 8, 9, 3, 8, 3, 1, 4])

10번 샘플링했기 때문에 고르게 샘플링되는 것 같지 않게 보임

그러니 샘플링 숫자를 늘리면 쉽게 확인 가능

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([116, 105,  95, 100,  84,  90,  97,  95, 107, 111]))

uniform의 사용법도 동일

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.07156624, 0.51330724, 0.78244744, 0.14237963, 0.05055468,
       0.13124955, 0.15801332, 0.99110938, 0.08459786, 0.92447632])

**예제: 결정트리 모델에서 min_impurity_decrease, max_depth, min_samples_split, min_samples_leaf 최적값 찾기**

min_samples_leaf: 리프 노드가 되기 위한 최소 샘플의 수

어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작으면 분할 안됨

샘플링 횟수는 n_tier 매개변수에 저장

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce351cc0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7cccce2f4610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce352da0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce353bb0>},
                   random_state=42)

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


최적의 모델은 이미 전체 훈련 세트로 훈련되어 best_estimator_에 저장됨

In [ ]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


**주의**

교차검증, 그리드 서치, 랜덤 서치는 모델을 훈련하는 것이 아니라 검증하는 것임

즉, 적절한 하이퍼파라미터를 찾는데 사용되어야 함

## 확인문제

In [ ]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce351cc0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7cccce2f4610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce352da0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce353bb0>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
